In [ ]:
# import py_entitymatching package
import py_entitymatching as em

In [17]:
# specify filepaths for tables A and B. 
path_A = '/Users/Hacker/Documents/Courses/Fall 2016/CS638/projects/CS638/stage3/formattedTableA.csv' # path_A is the file path where table_A.csv is stored
path_B = '/Users/Hacker/Documents/Courses/Fall 2016/CS638/projects/CS638/stage3/formattedTableB.csv' # path_B is the file path where table_B.csv is stored

In [18]:
# read table A; table A has 'ID' as the key attribute
A = em.read_csv_metadata(path_A, key='id')

# read table B; table B has 'ID' as the key attribute
B = em.read_csv_metadata(path_B, key='id')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [19]:
# Perform Attribute equivalence blocking on publishedYear, publishedMonth, publishedDay, pages
# Essentially multi-attributes equivalence blocking in BlackBoxBlocker
bb = em.BlackBoxBlocker()

In [20]:
# Detect the inconsistent type issue of "publishedYear", perform the corresponding type conversion so er
A['publishedYear'] = A['publishedYear'].convert_objects(convert_numeric=True)

/Users/Hacker/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


In [21]:
# Define the black_box function used to check the equivalence between numerical attributes
def numeric_value_block(x, y):
    x_day = x['publishedDay']
    y_day = y['publishedDay']
    x_month = x['publishedMonth']
    y_month = y['publishedMonth']
    x_year = x['publishedYear']
    y_year = y['publishedYear']
    x_pages = x['pages']
    y_pages = y['pages']
    day_diff = abs(x_day-y_day)
    month_diff = abs(x_month - y_month)
    year_diff = abs(x_year - y_year)
    pages_diff = abs(x_pages - y_pages)
    if day_diff > 2 or month_diff > 2 or year_diff > 2 or pages_diff > 10:
        return True
    else:
        return False

In [ ]:
# Pass the numeric_value_block to black box function
bb.set_black_box_function(numeric_value_block)
Cnumeric = bb.block_tables(A, B, l_output_attrs=['title', 'authors', 'publisher', 'ISBN13','publishedYear','publishedMonth','pages'], r_output_attrs=['title', 'authors', 'publisher', 'ISBN13','publishedYear','publishedMonth','pages'], l_output_prefix='ltable_', r_output_prefix='rtable_', verbose=False, show_progress=True, n_jobs=4)

In [9]:
# Checking the output of the numeric blocker
Cnumeric

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_publisher,ltable_ISBN13,ltable_publishedYear,ltable_publishedMonth,ltable_pages,rtable_title,rtable_authors,rtable_publisher,rtable_ISBN13,rtable_publishedYear,rtable_publishedMonth,rtable_pages
0,0,0,11,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,"Java Server Programming For Professionals, 2nd Edition Covers Java Ee 5 (Book/Cd Rom)","Ivan Bayross,Sharanam Shah,Cynthia Bayross,Vaishali Shah",NaN,9.788184e+12,NaN,NaN,NaN
1,1,0,66,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Client/Server Programming With Java And Corba,Robert Orfali,John Wiley & Sons,9.780471e+12,NaN,NaN,NaN
2,2,0,145,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,JAVA PROGRAMMING Vol.1,วีระศักดิ์ ซึงถาวร,NaN,NaN,NaN,NaN,NaN
3,3,0,189,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,On Becoming a Dinosaur: A Typesetter Speaks,Java Davis,Java Davis,NaN,NaN,NaN,NaN
4,4,0,224,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Programming In Java 2,K. Somasundaram,NaN,9.788180e+12,NaN,NaN,NaN
5,5,0,280,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Java,"Kathy Sierra,Bert Bates,Marie-Cécile Baland",NaN,9.782842e+12,NaN,NaN,NaN
6,6,0,301,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Java: A Beginner's Guide (Beginner's Guide),Herbert Schildt,McGraw-Hill/Osborne Media,9.780072e+12,NaN,NaN,NaN
7,7,0,438,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Java/J2ee,Shivprasad Koirala,NaN,9.788183e+12,NaN,NaN,NaN
8,8,0,445,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,The Java Virtual Machine Specification (Java),"Frank Yellin,Gilad Bracha",NaN,9.780321e+12,NaN,NaN,NaN
9,9,0,486,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Core Java Interview Questions You'll Most Likely Be Asked,Vibrant Publishers,NaN,NaN,NaN,NaN,NaN


In [10]:
def isbn_partial_block(x,y):
    try:
        isbn_A = str(int(x['ISBN13']))
    except Exception:
        isbn_A = "0000000000000"
  
    isbn3_head_A = isbn_A[0:4]
    
    try:
        isbn_B = str(int(y['ISBN13']))
    except Exception:
        isbn_B = "0000000000000"
    
    isbn3_head_B = isbn_B[0:4]
    
    if(isbn3_head_A != isbn3_head_B):
        return True
    else:
        return False
    

In [11]:
bb.set_black_box_function(isbn_partial_block)
Cisbn = bb.block_tables(A, B, l_output_attrs=['title', 'authors', 'publisher', 'ISBN13','publishedYear', 'publishedMonth', 'pages'], r_output_attrs=['title', 'authors', 'publisher', 'ISBN13', 'publishedYear', 'publishedMonth','pages'], l_output_prefix='ltable_', r_output_prefix='rtable_', verbose=True, show_progress=True, n_jobs=4)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:30:11


In [23]:
Cisbn

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_publisher,ltable_ISBN13,ltable_publishedYear,ltable_publishedMonth,ltable_pages,rtable_title,rtable_authors,rtable_publisher,rtable_ISBN13,rtable_publishedYear,rtable_publishedMonth,rtable_pages
0,0,0,0,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,The Java Language Specification (The Java Series),"James Gosling,Bill Joy,Guy L. Steele Jr.,Gilad Bracha",Addison-Wesley Professional,9.780321e+12,2005.0,6.0,651.0
1,1,0,1,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Java Performance Tuning,Jack Shirazi,O'Reilly Media,9.780596e+12,2003.0,1.0,600.0
2,2,0,2,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Java Software Solutions (Java 5.0 version): Foundations of Program Design,"John Lewis,William Loftus",Addison Wesley Publishing Company,9.780321e+12,2005.0,7.0,944.0
3,3,0,3,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Data Structures and Abstractions with Java,"Frank M. Carrano,Walter J. Savitch",Prentice Hall,9.780132e+12,2006.0,8.0,998.0
4,4,0,4,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Java and XSLT,Eric M. Burke,O'Reilly Media,9.780596e+12,2001.0,9.0,528.0
5,5,0,5,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Java: An Introduction to Problem Solving and Programming,Walter J. Savitch,Prentice Hall,9.780131e+12,2004.0,12.0,1060.0
6,6,0,6,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,"Core Java 2, Volume I--Fundamentals (Core Series)","Cay S. Horstmann,Gary Cornell",Prentice Hall,9.780131e+12,2004.0,8.0,784.0
7,7,0,8,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Just Java 2 [With CDROM],Peter van der Linden,Prentice Hall PTR,9.780130e+12,2001.0,12.0,1098.0
8,8,0,12,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Programming in Java,"Sachin Malhotra,Saurabh Choudhary","Oxford University Press, USA",9.780198e+12,2011.0,2.0,656.0
9,9,0,14,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Sun Performance and Tuning: Java and the Internet,"Adrian Cockcroft,Sun Microsystems Press",Prentice Hall,9.780131e+12,1998.0,4.0,624.0


In [28]:
C = em.combine_blocker_outputs_via_union([Cnumeric, Cisbn])

In [29]:
C

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_publisher,ltable_ISBN13,ltable_publishedYear,ltable_publishedMonth,ltable_pages,rtable_title,rtable_authors,rtable_publisher,rtable_ISBN13,rtable_publishedYear,rtable_publishedMonth,rtable_pages
0,0,0,0,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,The Java Language Specification (The Java Series),"James Gosling,Bill Joy,Guy L. Steele Jr.,Gilad Bracha",Addison-Wesley Professional,9.780321e+12,2005.0,6.0,651.0
1,1,0,1,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Java Performance Tuning,Jack Shirazi,O'Reilly Media,9.780596e+12,2003.0,1.0,600.0
2,2,0,2,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Java Software Solutions (Java 5.0 version): Foundations of Program Design,"John Lewis,William Loftus",Addison Wesley Publishing Company,9.780321e+12,2005.0,7.0,944.0
3,3,0,3,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Data Structures and Abstractions with Java,"Frank M. Carrano,Walter J. Savitch",Prentice Hall,9.780132e+12,2006.0,8.0,998.0
4,4,0,4,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Java and XSLT,Eric M. Burke,O'Reilly Media,9.780596e+12,2001.0,9.0,528.0
5,5,0,5,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Java: An Introduction to Problem Solving and Programming,Walter J. Savitch,Prentice Hall,9.780131e+12,2004.0,12.0,1060.0
6,6,0,6,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,"Core Java 2, Volume I--Fundamentals (Core Series)","Cay S. Horstmann,Gary Cornell",Prentice Hall,9.780131e+12,2004.0,8.0,784.0
7,7,0,8,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Just Java 2 [With CDROM],Peter van der Linden,Prentice Hall PTR,9.780130e+12,2001.0,12.0,1098.0
8,8,0,11,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,"Java Server Programming For Professionals, 2nd Edition Covers Java Ee 5 (Book/Cd Rom)","Ivan Bayross,Sharanam Shah,Cynthia Bayross,Vaishali Shah",NaN,9.788184e+12,NaN,NaN,NaN
9,9,0,12,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,Programming in Java,"Sachin Malhotra,Saurabh Choudhary","Oxford University Press, USA",9.780198e+12,2011.0,2.0,656.0


In [26]:
feature_table = em.get_features_for_blocking(A,B)

In [27]:
feature_table

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,id_id_exm,id,id,None,None,exact_match,<function id_id_exm at 0x11808e378>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,id_id_anm,id,id,None,None,abs_norm,<function id_id_anm at 0x11854be18>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,id_id_lev_dist,id,id,None,None,lev_dist,<function id_id_lev_dist at 0x11854bea0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,id_id_lev_sim,id,id,None,None,lev_sim,<function id_id_lev_sim at 0x11854bf28>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,title_title_jac_qgm_3_qgm_3,title,title,qgm_3,qgm_3,jaccard,<function title_title_jac_qgm_3_qgm_3 at 0x118560598>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,title_title_cos_dlm_dc0_dlm_dc0,title,title,dlm_dc0,dlm_dc0,cosine,<function title_title_cos_dlm_dc0_dlm_dc0 at 0x118560048>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,title_title_mel,title,title,None,None,monge_elkan,<function title_title_mel at 0x118560158>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,title_title_lev_dist,title,title,None,None,lev_dist,<function title_title_lev_dist at 0x1185601e0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,title_title_lev_sim,title,title,None,None,lev_sim,<function title_title_lev_sim at 0x118560268>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,authors_authors_jac_qgm_3_qgm_3,authors,authors,qgm_3,qgm_3,jaccard,<function authors_authors_jac_qgm_3_qgm_3 at 0x118560400>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [30]:
title_blocker=em.RuleBasedBlocker()

In [31]:
title_blocker.add_rule(['title_title_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.5'], feature_table)

'_rule_0'

In [32]:
Ctitle = title_blocker.block_candset(C, verbose=False, show_progress=True, n_jobs=4)

0%                          100%
[                              ]0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:05:19


In [34]:
Ctitle

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_publisher,ltable_ISBN13,ltable_publishedYear,ltable_publishedMonth,ltable_pages,rtable_title,rtable_authors,rtable_publisher,rtable_ISBN13,rtable_publishedYear,rtable_publishedMonth,rtable_pages
31,31,0,54,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,The Java Tutorial: A Short Course on the Basics,Kathy Walrath,Addison-Wesley Professional,9.780321e+12,2006.0,12.0,672.0
260,260,0,443,The Java Tutorial: A Short Course on the Basics / Edition 6,"Raymond Gallardo,Scott Hommel,Sowmya Kannan,Joni Gordon,Sharon Biocca Zakhour",Addison-Wesley,9780134034089,2014.0,12,864.0,The Java Tutorial: A Short Course on the Basics,"Sharon Zakhour,Sowmya Kannan,Raymond Gallardo",Addison-Wesley Professional,9.780133e+12,2013.0,2.0,713.0
7382,7382,5,441,Iron-Clad Java: Building Secure Web Applications,"Jim Manico,August Detlefsen",McGraw-Hill Professional Publishing,9780071835886,2014.0,9,304.0,Iron-Clad Java: Building Secure Web Applications,"Jim Manico,August Detlefsen",McGraw-Hill Education,9.780072e+12,2014.0,9.0,304.0
9166,9166,6,273,Java SE 8 for the Really Impatient / Edition 1,Cay S. Horstmann,Addison-Wesley,9780321927767,2014.0,1,240.0,Java SE 8 for the Really Impatient,Cay S. Horstmann,Addison-Wesley Professional,9.780322e+12,2014.0,1.0,215.0
16974,16974,12,366,Think Java: How to Think Like a Computer Scientist,"Downey,Mayfield","O'Reilly Media, Incorporated",9781491929568,2016.0,6,252.0,How to Think Like a Computer Scientist: Java Programming,Allen B. Downey,Createspace,9.781441e+12,2009.0,3.0,310.0
25001,25001,19,482,Doing Java: An Anthropological Detective Story,Niels Mulder,ATF Press,9789792111491,2006.0,12,245.0,Doing Java: An Anthropological Detective Story,Niels Mulder,Kanisius,9.789792e+12,2006.0,12.0,239.0
25484,25484,20,329,Java: How to Program / Edition 2,"Harvey M. Deitel,Paul J. Deitel",Prentice Hall Professional Technical Reference,9780138993948,1997.0,8,1063.0,Java: How to Program,"Harvey M. Deitel,Paul J. Deitel",Prentice Hall,9.780131e+12,2004.0,8.0,1568.0
29199,29199,22,254,Java 2 Network Security,"Marco Pistoia,Ashok K. Ramani,Milind Nagnur,Deepak Gupta,Duane F. Reller",Pearson Education,9780130155924,1999.0,8,713.0,Java Security,Scott Oaks,O'Reilly Media,9.780596e+12,2001.0,5.0,620.0
29939,29939,22,1813,Java 2 Network Security,"Marco Pistoia,Ashok K. Ramani,Milind Nagnur,Deepak Gupta,Duane F. Reller",Pearson Education,9780130155924,1999.0,8,713.0,Java Network Security,Robert S. MacGregor,Prentice Hall,9.780138e+12,1998.0,1.0,232.0
30951,30951,23,16,Sams Teach Yourself Java 6 in 21 Days,"Rogers Cadenhead,Laura Lemay",Sams,9780672329432,2007.0,6,720.0,Sams Teach Yourself Java in 24 Hours,Rogers Cadenhead,Sams,9.780672e+12,2009.0,11.0,432.0


In [35]:
# save the table C to csv file
em.to_csv_metadata(Ctitle, './tableC.csv')

File already exists at ./tableC.csv; Overwriting it
Metadata file already exists at ./tableC.metadata. Overwriting it


True

In [36]:
# specify the file path for table A and table B
path_A = "/Users/Hacker/Documents/Courses/Fall 2016/CS638/projects/CS638/stage4/formattedTableA.csv"
path_B = "/Users/Hacker/Documents/Courses/Fall 2016/CS638/projects/CS638/stage4/formattedTableB.csv"

In [37]:
# specify the file path for table C got from blocking stage
path_C = "/Users/Hacker/Documents/Courses/Fall 2016/CS638/projects/CS638/stage4/tableC.csv"

In [38]:
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [39]:
# read table C
C = em.read_csv_metadata(path_C, ltable=A, rtable=B)

In [40]:
sample_table = em.sample_table(C,400,replace=False,verbose=False)

In [61]:
# sample 400 tuple pairs from table C
sample_table

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_publisher,ltable_ISBN13,rtable_title,rtable_authors,rtable_publisher,rtable_ISBN13
3,2140,1,1852,Java : Complete Java Programming Guide.,Harry. H. Chaudhary.,Harry & Associates.,2940046303308,Java: : Learn Complete Java Programming with Fun.,Harry H. Chaudhary,Createspace,9.781502e+12
5,10511,6,273,Java SE 8 for the Really Impatient / Edition 1,Cay S. Horstmann,Addison-Wesley,9780321927767,Java SE 8 for the Really Impatient,Cay S. Horstmann,Addison-Wesley Professional,9.780322e+12
23,54393,32,1906,Teach Yourself Internet Game Programming with Java in 21 Days,Michael Morrison,Sams,9781575211480,Teach Yourself Internet Game Programming with Java in 21 Days: With CDROM,Michael Morrison,Sams,9.781575e+12
29,60846,36,832,Professional Java Security,GARMS,Apress,9781861004253,Professional Java Soap,Henry Bequet,Wrox Press,9.781861e+12
35,69522,42,387,Java for Programmers,Paul Deitel,Pearson Education,9780132821551,Java for C/C++ Programmers,Michael C. Daconta,Wiley,9.780471e+12
36,69543,42,426,Java for Programmers,Paul Deitel,Pearson Education,9780132821551,C for Java Programmers,Tomasz Muldner,Pearson,9.780202e+12
42,77403,46,427,Java 2 For Dummies,Barry Burd,Wiley,9780764568589,Java & XML for Dummies,Barry Burd,John Wiley & Sons,9.780765e+12
84,169934,100,1297,Unified Software Engineering with Java / Edition 1,"Georges G. Merx,Ronald J. Norman",Prentice Hall,9780130473769,Unified Software Engineering with Java,Georges G. Merx,Prentice Hall,9.780130e+12
93,185960,110,577,Advanced Java,Chris Laffra,Prentice Hall Professional Technical Reference,9780135343487,Advanced Java,Chris Laffra,Prentice Hall,9.780135e+12
114,228125,133,1045,Compiling with C# and Java,Pat D. Terry,Longman Publishing Group,9780321263605,Compiling with C# and Java,Pat D. Terry,Addison Wesley Publishing Company,9.780321e+12


In [43]:
# label the sample tuple pairs
gold_table = em.label_table(sample_table,'gold',verbose=False)

Column name (gold) is not present in dataframe


In [44]:
# save the tableG to csv file
em.to_csv_metadata(gold_table, './tableG.csv')

True